# Language modelling

The exercise shows how a language model may be used to solve word-prediction tasks and to generate text.


## Tasks

1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.
2. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository. 

3. Produce the predictions for the following sentences (use each model and check 5 predictions):


    i. (M) Warszawa to największe `[MASK]`.
    ii. (D) Te zabawki należą do `[MASK]`.
    iii. (C) Policjant przygląda się `[MASK]`.
    iv. (B) Na środku skrzyżowania widać `[MASK]`.
    v. (N) Właściciel samochodu widział złodzieja z `[MASK]`.
    vi. (Ms) Prezydent z premierem rozmawiali wczoraj o `[MASK]`.
    vii. (W) Witaj drogi `[MASK]`.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [27]:
from transformers import AutoTokenizer, AutoModel
import torch
from transformers import *


In [121]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)


In [31]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")

sequence = f"Warszawa to największe {tokenizer.mask_token}."



Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
predict(sequence)

['Warszawa to największe miasto.',
 'Warszawa to największe lotnisko.',
 'Warszawa to największe centrum.',
 'Warszawa to największe miasta.',
 'Warszawa to największe atrakcje.']

In [123]:
class Model:
    def __init__(self):
        self.df = pd.DataFrame()
        pass
    
    def predict(self,sequence):
        pass
    
    def preprocess(self,sequences):
        for i, sequence in enumerate(sequences):
            self.df[f'{i}'] = self.predict(sequence)
            
    @staticmethod 
    def compare(model1, model2, model3,i):
        i = str(i)
        header = [model1.name,model2.name,model3.name]
        return pd.DataFrame(list(zip(model1.df[i], model2.df[i],model3.df[i])),columns=header)


In [105]:



class AllegroModel(Model):
    def __init__(self):
        super().__init__()
        self.name = 'allegro/herbert-base-cased'
        self.tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
        self.model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")


    def predict(self,sequence):
        sequence = sequence.replace("[MASK]",self.tokenizer.mask_token)
        inputs = self.tokenizer(sequence, return_tensors="pt")
        mask_token_index = torch.where(inputs["input_ids"] == self.tokenizer.mask_token_id)[1]
        token_logits = self.model(**inputs).logits

        mask_token_logits = token_logits[0, mask_token_index, :]
        top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

        result = [] 
        for token in top_5_tokens:
            result.append(sequence.replace(self.tokenizer.mask_token, self.tokenizer.decode([token])))
        return result
            
model = AllegroModel()
model.predict(f"Adam Mickiewicz wielkim polskim [MASK] był.")

sequences = ["Adam Mickiewicz wielkim polskim [MASK] był.",f"Stolicą Polski jest [MASK]."]
model.preprocess(sequences)
model.df

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,0,1
0,Adam Mickiewicz wielkim polskim poetą był.,Stolicą Polski jest Warszawa.
1,Adam Mickiewicz wielkim polskim pisarzem był.,Stolicą Polski jest Kraków.
2,Adam Mickiewicz wielkim polskim politykiem był.,Stolicą Polski jest Gdańsk.
3,Adam Mickiewicz wielkim polskim nie był.,Stolicą Polski jest Poznań.
4,Adam Mickiewicz wielkim polskim człowiekiem był.,Stolicą Polski jest Wrocław.


In [106]:
            
class BertModel(Model):
    def __init__(self):
        super().__init__()
        self.name = 'Geotrend/bert-base-pl-cased'

        self.model = BertForMaskedLM.from_pretrained('Geotrend/bert-base-pl-cased')
        self.tokenizer = BertTokenizer.from_pretrained('Geotrend/bert-base-pl-cased')
        self.nlp = pipeline('fill-mask', model=self.model, tokenizer=self.tokenizer)
    
    def predict(self,sequence):
        sequence = sequence.replace("[MASK]",self.nlp.tokenizer.mask_token)
        
        result =self.nlp(sequence)
        out = []
        for pred in result:
            out.append(f"{pred['sequence']}:{np.round(pred['score'],2)}")
#             print(pred) 
        return out
    
model = BertModel()
model.predict(f"Adam Mickiewicz wielkim polskim [MASK] był.")

sequences = ["Adam Mickiewicz wielkim polskim [MASK] był.",f"Stolicą Polski jest [MASK]."]
model.preprocess(sequences)
model.df

,0,1
0,Adam Mickiewicz wielkim polskim polskim był.:0.22,Stolicą Polski jest RP.:0.1
1,Adam Mickiewicz wielkim polskimiem był.:0.22,Stolicą Polski jest Rzeczypospolitej.:0.06
2,Adam Mickiewicz wielkim polskim autorem był.:0.08,Stolicą Polski jest woj.:0.04
3,Adam Mickiewicz wielkim polskim synem był.:0.02,Stolicą Polski jest Polski.:0.02
4,Adam Mickiewicz wielkim polskim Polaków był.:0.02,Stolicą Polski jest r.:0.02


In [107]:

                               
class KleczekModel(Model):
    def __init__(self):
        super().__init__()
        self.name = 'dkleczek/bert-base-polish-cased-v1'
        self.model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
        self.tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
        self.nlp = pipeline('fill-mask', model=self.model, tokenizer=self.tokenizer)

    def predict(self,sequence):
        sequence = sequence.replace("[MASK]",self.nlp.tokenizer.mask_token)
        
        result =self.nlp(sequence)
        out = []
        for pred in result:
            out.append(f"{pred['sequence']}:{np.round(pred['score'],2)}")
#             print(pred) 
        return out
            
model = KleczekModel()
model.predict(f"Adam Mickiewicz wielkim polskim [MASK] był.")

sequences = ["Adam Mickiewicz wielkim polskim [MASK] był.",f"Stolicą Polski jest [MASK]."]
model.preprocess(sequences)
model.df

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,0,1
0,Adam Mickiewicz wielkim polskim pisarzem był.:...,Stolicą Polski jest Warszawa.:0.25
1,Adam Mickiewicz wielkim polskim człowiekiem by...,Stolicą Polski jest Gdańsk.:0.06
2,Adam Mickiewicz wielkim polskim bohaterem był....,Stolicą Polski jest Polska.:0.05
3,Adam Mickiewicz wielkim polskim mistrzem był.:...,Stolicą Polski jest Poznań.:0.05
4,Adam Mickiewicz wielkim polskim artystą był.:0.03,Stolicą Polski jest Ukraina.:0.04


In [108]:
allegroModel = AllegroModel()
kleczekModel = KleczekModel()
bertModel = BertModel()

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

In [128]:
sequences = ["Warszawa to największe [MASK].","Te zabawki należą do [MASK].","Policjant przygląda się [MASK].","Na środku skrzyżowania widać [MASK].","Właściciel samochodu widział złodzieja z [MASK].","Prezydent z premierem rozmawiali wczoraj o [MASK].","Witaj drogi [MASK]."]

allegroModel.preprocess(sequences)
kleczekModel.preprocess(sequences)
bertModel.preprocess(sequences)

for i in range(len(sequences)):
    df = Model.compare(allegroModel, kleczekModel, bertModel,i)
    display(df)

,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Warszawa to największe miasto.,Warszawa to największe miasto.:0.96,Warszawa to największe miasto.:0.32
1,Warszawa to największe lotnisko.,Warszawa to największe województwo.:0.01,Warszawa to największe miasta.:0.09
2,Warszawa to największe centrum.,Warszawa to największe lotnisko.:0.01,Warszawa to największe woj.:0.07
3,Warszawa to największe miasta.,Warszawa to największe miasteczko.:0.0,Warszawa to największe Warszawa.:0.04
4,Warszawa to największe atrakcje.,Warszawa to największe państwo.:0.0,Warszawa to największe miast.:0.04


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Te zabawki należą do rodziny.,Te zabawki należą do ciebie.:0.19,Te zabawki należą do tzw.:0.2
1,Te zabawki należą do nas.,Te zabawki należą do mnie.:0.19,Te zabawki należą do pt.:0.06
2,Te zabawki należą do nich.,Te zabawki należą do nas.:0.05,Te zabawki należą do pl.:0.06
3,Te zabawki należą do najlepszych.,Te zabawki należą do pana.:0.03,Te zabawki należą do odc.:0.03
4,Te zabawki należą do ..,Te zabawki należą do niego.:0.03,Te zabawki należą do ok.:0.03


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Policjant przygląda się mężczyźnie.,Policjant przygląda się temu.:0.16,Policjant przygląda się go.:0.07
1,Policjant przygląda się kobiecie.,Policjant przygląda się sprawie.:0.11,Policjant przygląda się ok.:0.05
2,Policjant przygląda się mu.,Policjant przygląda się im.:0.04,Policjant przygląda się się.:0.03
3,Policjant przygląda się dziewczynie.,Policjant przygląda się wszystkiemu.:0.04,Policjant przygląda się pt.:0.02
4,Policjant przygląda się sprawie.,Policjant przygląda się panu.:0.04,Policjant przygląda się nie.:0.02


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Na środku skrzyżowania widać rondo.,Na środku skrzyżowania widać rzekę.:0.06,Na środku skrzyżowania widać św.:0.07
1,Na środku skrzyżowania widać samochody.,Na środku skrzyżowania widać ulicę.:0.05,Na środku skrzyżowania widać ok.:0.05
2,Na środku skrzyżowania widać radiowóz.,Na środku skrzyżowania widać drzewa.:0.03,Na środku skrzyżowania widać obraz.:0.05
3,Na środku skrzyżowania widać samochód.,Na środku skrzyżowania widać drogę.:0.03,Na środku skrzyżowania widać rok.:0.03
4,Na środku skrzyżowania widać wiadukt.,Na środku skrzyżowania widać las.:0.02,Na środku skrzyżowania widać kościół.:0.02


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Właściciel samochodu widział złodzieja z samochodu.,Właściciel samochodu widział złodzieja z bronią.:0.18,Właściciel samochodu widział złodzieja z pt.:0.22
1,Właściciel samochodu widział złodzieja z włamaniem.,Właściciel samochodu widział złodzieja z tyłu.:0.08,Właściciel samochodu widział złodzieja z tzw.:0.06
2,Właściciel samochodu widział złodzieja z auta.,Właściciel samochodu widział złodzieja z ulicy.:0.04,Właściciel samochodu widział złodzieja z ul.:0.03
3,Właściciel samochodu widział złodzieja z kierowcą.,Właściciel samochodu widział złodzieja z bliska.:0.03,Właściciel samochodu widział złodzieja z woj.:0.02
4,Właściciel samochodu widział złodzieja z parkingu.,Właściciel samochodu widział złodzieja z zewnątrz.:0.02,Właściciel samochodu widział złodzieja z kościoła.:0.02


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Prezydent z premierem rozmawiali wczoraj o przyszłości.,Prezydent z premierem rozmawiali wczoraj o tym.:0.18,Prezydent z premierem rozmawiali wczoraj o północy.:0.22
1,Prezydent z premierem rozmawiali wczoraj o Polsce.,Prezydent z premierem rozmawiali wczoraj o Polsce.:0.04,Prezydent z premierem rozmawiali wczoraj o tym.:0.14
2,Prezydent z premierem rozmawiali wczoraj o bezpieczeństwie.,Prezydent z premierem rozmawiali wczoraj o budżecie.:0.04,Prezydent z premierem rozmawiali wczoraj o nim.:0.06
3,Prezydent z premierem rozmawiali wczoraj o polityce.,Prezydent z premierem rozmawiali wczoraj o ASF.:0.04,Prezydent z premierem rozmawiali wczoraj o co.:0.03
4,Prezydent z premierem rozmawiali wczoraj o Warszawie.,Prezydent z premierem rozmawiali wczoraj o ustawie.:0.03,Prezydent z premierem rozmawiali wczoraj o św.:0.03


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Witaj drogi Łukasz.,Witaj drogi chłopcze.:0.11,Witaj drogi wyd.:0.04
1,Witaj drogi Boże.,Witaj drogi przyjacielu.:0.09,Witaj drogi op.:0.04
2,Witaj drogi człowieku.,Witaj drogi bracie.:0.04,Witaj drogi pt.:0.02
3,Witaj drogi Karol.,Witaj drogi kolego.:0.04,Witaj drogi ps.:0.01
4,Witaj drogi Marcin.,Witaj drogi synu.:0.03,Witaj drogi gen.:0.01


   
4. Check the model predictions for the following sentences (using each model):


    i. Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie `[MASK]`.
    ii. Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie `[MASK]`.
   

In [129]:
sequences = ["Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK]."]

allegroModel.preprocess(sequences)
kleczekModel.preprocess(sequences)
bertModel.preprocess(sequences)

for i in range(len(sequences)):
    df = Model.compare(allegroModel, kleczekModel, bertModel,i)
    display(df)


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie poddał.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził.:0.17","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się niewiedział.:0.1"
1,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zdziwił.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie bał.:0.07","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie było.:0.09"
2,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział.:0.07","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie stało.:0.05"
3,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zastanawiał.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie pojawił.:0.06","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się niedził.:0.04"
4,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie przyznał.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zabił.:0.06","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie stanie.:0.03"


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła.:0.3","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie było.:0.1"
1,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie przyznała.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała.:0.15","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się niewia.:0.05"
2,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała.:0.11","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie stało.:0.04"
3,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zmieniła.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie pojawiła.:0.08","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nieśli.:0.03"
4,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zabiła.:0.07","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się niewiedział.:0.03"


5. Check the model predictions for the following sentences:


    i. `[MASK]` wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
    ii. W wakacje odwiedziłem `[MASK]`, który jest stolicą Islandii.
    iii. Informatyka na `[MASK]` należy do najlepszych kierunków w Polsce.
   

In [130]:
sequences = ["[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","W wakacje odwiedziłem [MASK], który jest stolicą Islandii.","Informatyka na [MASK] należy do najlepszych kierunków w Polsce."]

allegroModel.preprocess(sequences)
kleczekModel.preprocess(sequences)
bertModel.preprocess(sequences)

for i in range(len(sequences)):
    df = Model.compare(allegroModel, kleczekModel, bertModel,i)
    display(df)

,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,"Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.17","Jego wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.05"
1,"Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","Mięso wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.06","Za wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.03"
2,"Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.06","Po wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.02"
3,"Następnie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","Nie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.06","Nie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.02"
4,"Ciało wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.","Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.04","Ich wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.:0.02"


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,"W wakacje odwiedziłem Kraków, który jest stolicą Islandii.","W wakacje odwiedziłem kraj, który jest stolicą Islandii.:0.29","W wakacje odwiedziłem Island, który jest stolicą Islandii.:0.33"
1,"W wakacje odwiedziłem Oslo, który jest stolicą Islandii.","W wakacje odwiedziłem Cypr, który jest stolicą Islandii.:0.05","W wakacje odwiedziłem Reykjavík, który jest stolicą Islandii.:0.07"
2,"W wakacje odwiedziłem Londyn, który jest stolicą Islandii.","W wakacje odwiedziłem Meksyk, który jest stolicą Islandii.:0.03","W wakacje odwiedziłem Porto, który jest stolicą Islandii.:0.04"
3,"W wakacje odwiedziłem Gdańsk, który jest stolicą Islandii.","W wakacje odwiedziłem Gibraltar, który jest stolicą Islandii.:0.03","W wakacje odwiedziłem miasto, który jest stolicą Islandii.:0.02"
4,"W wakacje odwiedziłem Toruń, który jest stolicą Islandii.","W wakacje odwiedziłem Wellington, który jest stolicą Islandii.:0.03","W wakacje odwiedziłem Port, który jest stolicą Islandii.:0.02"


,allegro/herbert-base-cased,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased
0,Informatyka na pewno należy do najlepszych kierunków w Polsce.,Informatyka na wsi należy do najlepszych kierunków w Polsce.:0.1,Informatyka na uczelni należy do najlepszych kierunków w Polsce.:0.07
1,Informatyka na AGH należy do najlepszych kierunków w Polsce.,Informatyka na świecie należy do najlepszych kierunków w Polsce.:0.09,Informatyka na rynku należy do najlepszych kierunków w Polsce.:0.07
2,Informatyka na UW należy do najlepszych kierunków w Polsce.,Informatyka na żywo należy do najlepszych kierunków w Polsce.:0.09,Informatyka na internet należy do najlepszych kierunków w Polsce.:0.06
3,Informatyka na studiach należy do najlepszych kierunków w Polsce.,Informatyka na pewno należy do najlepszych kierunków w Polsce.:0.07,Informatyka na świecie należy do najlepszych kierunków w Polsce.:0.05
4,Informatyka na UMK należy do najlepszych kierunków w Polsce.,Informatyka na odległość należy do najlepszych kierunków w Polsce.:0.04,Informatyka na terenie należy do najlepszych kierunków w Polsce.:0.04


6. If you want to use causal language models such as PapuGaPT2 or plT5, you should change the last three examples to accomodate for the fact, that these
   models are better suited for causal language modelling.
   

7. Answer the following questions:
   
   
    i. Which of the models produced the best results?
    ii. Was any of the models able to capture Polish grammar?
    iii. Was any of the models able to capture long-distant relationships between the words?
    iv. Was any of the models able to capture world knowledge?
    v. What are the most striking errors made by the models?